In [1]:
import numpy as np
import plotly.graph_objects as go
import sys
sys.path.append('../../assets/python/')
import tfb
from dice_roller import *
from icepool import d

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

COLOR_LIST = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]

In [2]:
# analytic functions
"""
m_save = 0.0

fail_state = {
    'probability': 0.65,
    'damage': '1d8',
    'mean': 4.5,
    'sigma': np.sqrt((8**2 - 1)/12),
}

save_state = {
    'probability': 1 - fail_state['probability'],
    'damage': '0',
    'mean': m_save*fail_state['mean'],
    'sigma': m_save*fail_state['sigma'],
}
"""
def save_mean(f_state, s_state, n_targets=1):
    return n_targets*(f_state['probability']*f_state['mean'] 
            + s_state['probability']*s_state['mean'])

def save_variance(f_state, s_state, n_targets=1):
    mean = save_mean(f_state, s_state, 1)
    var = (   f_state['probability']*f_state['sigma']**2 
            + s_state['probability']*s_state['sigma']**2
            + f_state['probability']*f_state['mean']**2 
            + s_state['probability']*s_state['mean']**2
            - mean**2)
    
    cov = (f_state['probability']*f_state['sigma'] + s_state['probability']*s_state['sigma'])**2
    
    return n_targets*var + n_targets*(n_targets - 1)*cov

def save_sigma(f_state, s_state, n_targets=1):
    return np.sqrt(save_variance(f_state, s_state, n_targets))

# icepool functions
def success_damage(factor, fail_damage_roll):
    """Converts the damage done on a failed save to damage on a successful save.
    """
    return np.floor(factor*fail_damage_roll)

def saving_throw_fails(save_bonus, save_roll, difficulty_class):
    """Used to determine the probability distribution of failed saves and successful ones.
    """
    if save_roll + save_bonus >= difficulty_class:
        return 0.0
    else:
        return 1.0

def save_damage(n_targets, targets_roll, fail_damage_roll, save_factor):
    """Used to convert the distribution of saving throw failures for a given number of targets into the total damage.
    """
    return fail_damage_roll*targets_roll + (n_targets - targets_roll)*success_damage(save_factor, fail_damage_roll)

def saving_throw_damage(save_bonus, save_roll, difficulty_class, fail_damage_roll, save_factor, n_targets):
    """Calculates the distribution of total damage done by the saving throw against a set number of targets.
    """
    st = n_targets @ map(saving_throw_fails, save_bonus, save_roll, difficulty_class)
    return map(save_damage, n_targets, st, fail_damage_roll, save_factor)


In [3]:
# Save damage mean and sigma vs fail probability for fixed damage outcomes

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=60, t=40, pad=4),
        xaxis=dict(
            title_text='failed save probability',
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
            range=[0,1],
        ),
        yaxis=dict(
            title_text='damage',
            tickformat='.1f',
            range=[0,1.2],
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
            bgcolor='rgba(0,0,0,0)',
            itemclick=False, itemdoubleclick=False,
        )
    )
)

def y_annotation_large(i):
    return 1.03 - i*0.061

def y_annotation_small(i):
    return 1.03 - i*0.095

p_fail = np.linspace(0,20,21)/20.0
i = 0
for m_s in [0, 0.5]:
    fail_state    = {'probability': 0, 'mean': 1.0, 'sigma': 0}
    success_state = {'probability': 0, 'mean': m_s, 'sigma': 0}

    mean = []
    sigma = []
    for p in p_fail:
        fail_state['probability'] = p
        success_state['probability'] = 1 - p
        mean  += [save_mean(fail_state, success_state)]
        sigma += [save_sigma(fail_state, success_state)]
    
    fig.add_trace(go.Scatter(
        x=p_fail,
        y=mean,
        name=' ',
        mode='lines',
        line=dict(color=COLOR_LIST[i]),
        hovertemplate=
            f'<b>{m_s:.0%} damage on save</b><br>'+
            'fail probability %{x:.2f}<br>'+
            'mean %{y:.2f}<extra></extra>'
    ))
    fig.add_annotation(
        name=f'{2*i+1:.0f}',
        xref='paper', xanchor='left',   x=0.09,
        yref='paper', yanchor='middle', y=y_annotation_large(2*i+1),
        showarrow=False,
        text=f'$m_{{\mathrm{{s}}}} = {m_s:.1f} - \mathrm{{mean}}$',
    )

    fig.add_trace(go.Scatter(
        x=p_fail,
        y=sigma,
        name=' ',
        mode='lines',
        line=dict(color=COLOR_LIST[i], dash='dash'),
        hovertemplate=
            f'<b>{m_s:.0%} damage on save</b><br>'+
            'fail probability %{x:.2f}<br>'+
            'std dev %{y:.2f}<extra></extra>'
    ))
    fig.add_annotation(
        name=f'{2*i+2:.0f}',
        xref='paper', xanchor='left',   x=0.09,
        yref='paper', yanchor='middle', y=y_annotation_large(2*i+2),
        showarrow=False,
        text=f'$m_{{\mathrm{{s}}}} = {m_s:.1f} - \mathrm{{sigma}}$',
    )

    i += 1

fig.add_trace(go.Scatter(
    x=[0.65, 0.65], 
    y=[0,3],
    mode='lines',
    showlegend=False,
    hoverinfo='skip',
    line=dict(color='black', dash='dash')
))

# show figure
fig.update_layout(width=600, height=450, barmode='group')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-fixed-save-normalized-damage-large', style='aspect-ratio:600/450')
    for i in range(1, 5):
        fig.update_annotations(selector=dict(name=f'{i:.0f}'), x=0.14, y=y_annotation_small(i))
    tfb.save_fig_html(fig, format='small', name=f'./fig-fixed-save-normalized-damage-small', style='aspect-ratio:600/500')

In [4]:
# Save damage CV vs fail probability for fixed damage outcomes

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=60, t=40, pad=4),
        xaxis=dict(
            title_text='failed save probability',
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
            range=[0,1],
        ),
        yaxis=dict(
            title_text='CV (sigma/mean)',
            tickformat='.1f',
            range=[0,3],
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.66, y=1.00,
            orientation='v',
            tracegroupgap=0,
            bgcolor='rgba(0,0,0,0)',
        )
    )
)

def y_annotation_large(i):
    return 1.03 - i*0.061

def y_annotation_small(i):
    return 1.03 - i*0.095

p_fail = np.linspace(0,20,21)/20.0
i = 0
for m_s in [0, 0.5]:
    fail_state = {
        'probability': 0,
        'mean': 1,
        'sigma': 0,
    }

    success_state = {
        'probability': 0,
        'mean': m_s,
        'sigma': 0,
    }

    cv = []
    for p in p_fail:
        fail_state['probability'] = p
        success_state['probability'] = 1 - p
        cv += [save_sigma(fail_state, success_state)/save_mean(fail_state, success_state)]
    
    fig.add_trace(go.Scatter(
        x=p_fail,
        y=cv,
        name=' ',
        #name=f'{m_s:.0%} damage on save',
        mode='lines',
        hovertemplate=
            f'<b>{m_s:.0%} damage on save</b><br>'+
            'fail probability %{x:.2f}<br>'+
            'action CV %{y:.2f}<extra></extra>'
    ))
    fig.add_annotation(
        name=f'{i:.0f}',
        xref='paper', xanchor='left',   x=0.75,
        yref='paper', yanchor='middle', y=y_annotation_large(i+1),
        showarrow=False,
        text=f'$m_{{\mathrm{{s}}}} = {m_s:.1f} $',
    )
    i += 1

fig.add_trace(go.Scatter(
    x=[0.65, 0.65], 
    y=[0,3],
    mode='lines',
    showlegend=False,
    hoverinfo='skip',
    line=dict(color='black', dash='dash')
))

# show figure
fig.update_layout(width=600, height=450, barmode='group')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-fixed-save-cv-large')
    for i in range(2):
        fig.update_annotations(selector=dict(name=f'{i:.0f}'), x=0.80, y=y_annotation_small(i+1))
    tfb.save_fig_html(fig, format='small', name=f'./fig-fixed-save-cv-small', style='aspect-ratio:600/500')

/var/folders/k4/phz5f3ss2_91t6j4s71jmp540000gn/T/ipykernel_35076/4044573080.py:54: RuntimeWarning:

invalid value encountered in scalar divide



In [26]:
# Save damage mean and sigma vs fail probability with m=0.5 and s=0.5

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=60, t=40, pad=4),
        xaxis=dict(
            title_text='failed save probability',
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
            range=[0,1],
        ),
        yaxis=dict(
            title_text='damage',
            tickformat='.1f',
            range=[0,1.2],
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
            bgcolor='rgba(0,0,0,0)',
        )
    )
)


p_fail = np.linspace(0,20,21)/20.0

m_s = 0.5
s_s = 0.5
cv_f = 0.0
fail_state = {'probability': 0, 'mean': 1, 'sigma': cv_f}
success_state = {'probability': 0, 'mean': m_s*fail_state['mean'], 'sigma': s_s*fail_state['sigma']}
mean = []
sigma_fixed = []
for p in p_fail:
    fail_state['probability'] = p
    success_state['probability'] = 1 - p
    mean        += [save_mean(fail_state, success_state)]
    sigma_fixed += [save_sigma(fail_state, success_state)]

m_s = 0.5
s_s = 0.5
cv_f = 0.3
fail_state = {'probability': 0, 'mean': 1, 'sigma': cv_f}
success_state = {'probability': 0, 'mean': m_s*fail_state['mean'], 'sigma': s_s*fail_state['sigma']}
sigma_var = []
sigma_total = []
for p, s in zip(p_fail, sigma_fixed):
    fail_state['probability'] = p
    success_state['probability'] = 1 - p
    sigma_total += [save_sigma(fail_state, success_state)]
    sigma_var += [np.sqrt(save_variance(fail_state, success_state) - s**2)]

fig.add_trace(go.Scatter(
    x=p_fail,
    y=mean,
    name=f'mean',
    mode='lines',
    hovertemplate=
        'fail probability %{x:.2f}<br>'+
        'mean %{y:.2f}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=p_fail,
    y=sigma_fixed,
    name=f'sigma - saving throw',
    mode='lines',
    hovertemplate=
        'fail probability %{x:.2f}<br>'+
        'sigma %{y:.2f}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=p_fail,
    y=sigma_var,
    name=f'sigma - damage roll',
    mode='lines',
    hovertemplate=
        'fail probability %{x:.2f}<br>'+
        'sigma %{y:.2f}<extra></extra>'
))


fig.add_trace(go.Scatter(
    x=p_fail,
    y=sigma_total,
    name=f'sigma - total',
    mode='lines',
    hovertemplate=
        'fail probability %{x:.2f}<br>'+
        'sigma %{y:.2f}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=[0.65, 0.65], 
    y=[0,3],
    mode='lines',
    showlegend=False,
    hoverinfo='skip',
    line=dict(color='black', dash='dash')
))

# show figure
fig.update_layout(width=600, height=450, barmode='group')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-save-normalized-damage-large', style='aspect-ratio:600/450')
    tfb.save_fig_html(fig, format='small', name=f'./fig-save-normalized-damage-small', style='aspect-ratio:600/500')

In [20]:
# Save damage CV vs fail probability with m=0.5 and s=0.5

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=60, t=40, pad=4),
        xaxis=dict(
            title_text='failed save probability',
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
            range=[0,1],
        ),
        yaxis=dict(
            title_text='CV (sigma/mean)',
            tickformat='.1f',
            range=[0,0.8],
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

def y_annotation_large(i):
    return 1.03 - i*0.061

def y_annotation_small(i):
    return 1.03 - i*0.095

p_fail = np.linspace(0,20,21)/20.0
m_s = 0.5
s_s = 0.5
i = 0
for cv in [0.1, 0.2, 0.3]:
    fail_state = {'probability': 0, 'mean': 1, 'sigma': cv}
    success_state = {'probability': 0, 'mean': m_s*fail_state['mean'], 'sigma': s_s*fail_state['sigma']}

    act_cv = []
    for p in p_fail:
        fail_state['probability'] = p
        success_state['probability'] = 1 - p
        act_cv += [save_sigma(fail_state, success_state)/save_mean(fail_state, success_state)]
    
    fig.add_trace(go.Scatter(
        x=p_fail,
        y=act_cv,
        name=' ',
        #name=f'fail CV = {cv:.2f}',
        mode='lines',
        hovertemplate=
            f'<b>fail CV = {cv:.2f}</b><br>'
            'fail probability %{x:.2f}<br>'+
            'action CV %{y:.2f}<extra></extra>'
    ))

    fig.add_annotation(
        name=f'{i:.0f}',
        xref='paper', xanchor='left',   x=0.09,
        yref='paper', yanchor='middle', y=y_annotation_large(i+1),
        showarrow=False,
        text=f'$CV_{{\mathrm{{f}}}} = {cv:.1f}$',
    )
    i += 1

fig.add_trace(go.Scatter(
    x=[0.65, 0.65], 
    y=[0,3],
    mode='lines',
    showlegend=False,
    hoverinfo='skip',
    line=dict(color='black', dash='dash')
))

# show figure
fig.update_layout(width=600, height=450, barmode='group')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-save-cv-large')
    for i in range(3):
        fig.update_annotations(selector=dict(name=f'{i:.0f}'), x=0.14, y=y_annotation_small(i+1))
    tfb.save_fig_html(fig, format='small', name=f'./fig-save-cv-small', style='aspect-ratio:600/500')

In [16]:
# Total CV or saving throw that deals no damage on a successful save

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='failed save probability',
            range=[0, 1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.05),
        ),
        yaxis=dict(
            title_text='CV (sigma/mean)',
            #type='log',
            range=[0, 3],
            #tick0=0, dtick=1,
            #minor=dict(tick0=0, dtick=1),
            tickformat='0.1f',
        ),
        legend=dict(
            xanchor='left', yanchor='bottom',
            x=0.00, y=0.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

p_fail = np.linspace(0.05, 1, 20)
ds = 0
for cv in [0.1,0.2,0.3]:
    fail_state = {'probability': 0, 'mean': 1, 'sigma': cv}
    success_state = {'probability': 0, 'mean': 0, 'sigma': 0}
    act_cv = []
    for p in p_fail:
        fail_state['probability'] = p
        success_state['probability'] = 1 - p
        act_cv += [save_sigma(fail_state, success_state)/save_mean(fail_state, success_state)]

    fig.add_trace(go.Scatter(
        x=p_fail, 
        y=act_cv,
        mode='lines',
        name=f'fail damage CV = {cv:.2f}',
        hovertemplate=
            'fail prob %{x:.0%}<br>'+
            'CV %{y:.2f}<extra></extra>'
    ))

fig.add_trace(go.Scatter(
    x=[0.65, 0.65], 
    y=[0,3],
    mode='lines',
    showlegend=False,
    hoverinfo='skip',
    line=dict(color='black', dash='dash')
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-save-cv-zero-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-save-cv-zero-small')

In [25]:
# Damage distribution for shatter against two targets
from icepool import map, d
import numpy as np

n_targets = 2
save_bns = -2
diff_cls = 12
fail_dmg = 3 @ d(8)
save_dmg_factor = 0.5
print(0.05*(diff_cls - save_bns - 1))

ad = n_targets @ fail_dmg
x_min = -1
x_max = int(ad.max_outcome() + 1)


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=60, t=40, pad=4),
        xaxis=dict(
            title_text='damage',
            range=[x_min+0.5,x_max-0.5],
            tick0=0, dtick=10,
            minor=dict(tick0=0, dtick=2.0),
        ),
        yaxis=dict(
            title_text='probability',
            tickformat='.0%',
            range=[-0.001, 0.11]
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

y_offset_step = 0.0
y_offset = 0.0

# one roll
name = f'1 damage roll'
ad = saving_throw_damage(save_bns, d(20), diff_cls, fail_dmg, save_dmg_factor, n_targets)

x_min = int(ad.min_outcome() - 1)
x_max = int(ad.max_outcome() + 1)
values = np.linspace(x_min, x_max, (x_max - x_min) + 1)
probabilities = np.array([ad.probability(v) for v in values])
fig.add_trace(go.Scatter(
    x=np.array(values), 
    y=np.array(probabilities) + y_offset,
    name=f'{name} (mean={ad.mean():.1f}, sigma={ad.sd():.1f})',
    mode='lines',
    line_shape='hvh',
    #fill='tozeroy',
    fill='tozeroy' if y_offset_step == 0 else 'toself',
    hoveron='points',
    customdata=np.array(probabilities),
    hovertemplate=
        f'<b>{name}</b><br>'+
        'damage %{x:.0f}<br>'+
        'probability %{customdata:.1%}<extra></extra>',
))
y_offset -= y_offset_step

# multiple roll
name = f'{n_targets:.0f} damage rolls'
ad = n_targets @ saving_throw_damage(save_bns, d(20), diff_cls, fail_dmg, save_dmg_factor, 1)

x_min = int(ad.min_outcome() - 1)
x_max = int(ad.max_outcome() + 1)
values = np.linspace(x_min, x_max, (x_max - x_min) + 1)
probabilities = np.array([ad.probability(v) for v in values])
fig.add_trace(go.Scatter(
    x=np.array(values), 
    y=np.array(probabilities) + y_offset,
    name=f'{name} (mean={ad.mean():.1f}, sigma={ad.sd():.1f})',
    mode='lines',
    line_shape='hvh',
    #fill='tozeroy',
    fill='tozeroy' if y_offset_step == 0 else 'toself',
    hoveron='points',
    customdata=np.array(probabilities),
    hovertemplate=
        f'<b>{name}</b><br>'+
        'damage %{x:.0f}<br>'+
        'probability %{customdata:.1%}<extra></extra>',
))
y_offset -= y_offset_step


# show figure
fig.update_layout(width=600, height=450, barmode='group')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-multi-target-example-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-multi-target-example-small', style='aspect-ratio:600/500')

0.65


In [23]:
# analytic solution to variance of saving throws with multiple targets (analytic)

def save_mean(f_state, s_state, n_targets=1):
    return n_targets*(f_state['probability']*f_state['mean'] 
            + s_state['probability']*s_state['mean'])

def save_variance(f_state, s_state, n_targets=1):
    mean = save_mean(f_state, s_state, 1)
    var = (   f_state['probability']*f_state['sigma']**2 
            + s_state['probability']*s_state['sigma']**2
            + f_state['probability']*f_state['mean']**2 
            + s_state['probability']*s_state['mean']**2
            - mean**2)
    
    cov = (f_state['probability']*f_state['sigma'] + s_state['probability']*s_state['sigma'])**2
    
    return n_targets*var + n_targets*(n_targets - 1)*cov

def save_sigma(f_state, s_state, n_targets=1):
    return np.sqrt(save_variance(f_state, s_state, n_targets))

m_save = 0.5

fail_state = {
    'probability': 0.65,
    'damage': '8d6',
    'mean': 8*3.5,
    'sigma': np.sqrt(8*(6**2 - 1)/12),
}

save_state = {
    'probability': 1 - fail_state['probability'],
    'damage': '0',
    'mean': m_save*fail_state['mean'],
    'sigma': m_save*fail_state['sigma'],
}

print(fail_state['sigma']/fail_state['mean'])
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=60, t=40, pad=4),
        xaxis=dict(
            title_text='targets',
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
            range=[0,11],
        ),
        yaxis=dict(
            title_text='CV (sigma/mean)',
            tickformat='.2f',
            range=[0,0.35],
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# single damage roll
targets = list(range(1, 11))
cv = [save_sigma(fail_state, save_state, n)/save_mean(fail_state, save_state, n) for n in targets]
fig.add_trace(go.Scatter(
    x=targets, 
    y=cv,
    name=f'single damage roll',
    mode='lines+markers',
    hovertemplate=
        f'<b>spell</b><br>'+
        'targets %{x:.0f}<br>'+
        'CV total %{y:.4f}<extra></extra>'
))

# multiple damage rolls
targets = list(range(1, 11))
cv = [(save_sigma(fail_state, save_state, 1)/save_mean(fail_state, save_state, 1))/np.sqrt(n) for n in targets]
fig.add_trace(go.Scatter(
    x=targets, 
    y=cv,
    name=f'multiple damage rolls',
    mode='lines+markers',
    hovertemplate=
        f'<b>spell</b><br>'+
        'targets %{x:.0f}<br>'+
        'CV total %{y:.4f}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=[0,11], 
    y=2*[fail_state['sigma']/fail_state['mean']],
    showlegend=False,
    mode='lines',
    line=dict(color='black', dash='dash'),
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450, barmode='group')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-multi-target-save-cv-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-multi-target-save-cv-small', style='aspect-ratio:600/500')

0.17251638983558856


In [152]:
# analytic solution to variance of saving throws with multiple targets (analytic)

def save_mean(f_state, s_state, n_targets=1):
    return n_targets*(f_state['probability']*f_state['mean'] 
            + s_state['probability']*s_state['mean'])

def save_variance(f_state, s_state, n_targets=1):
    mean = save_mean(f_state, s_state, 1)
    var = (   f_state['probability']*f_state['sigma']**2 
            + s_state['probability']*s_state['sigma']**2
            + f_state['probability']*f_state['mean']**2 
            + s_state['probability']*s_state['mean']**2
            - mean**2)
    
    cov = (f_state['probability']*f_state['sigma'] + s_state['probability']*s_state['sigma'])**2
    
    return n_targets*var + n_targets*(n_targets - 1)*cov

def save_sigma(f_state, s_state, n_targets=1):
    return np.sqrt(save_variance(f_state, s_state, n_targets))

m_save = 0.0

fail_state = {
    'probability': 0.65,
    'damage': '2d8',
    'mean': 2*4.5,
    'sigma': np.sqrt(2*(8**2 - 1)/12),
}

save_state = {
    'probability': 1 - fail_state['probability'],
    'damage': '0',
    'mean': m_save*fail_state['mean'],
    'sigma': m_save*fail_state['sigma'],
}

print(fail_state['sigma']/fail_state['mean'])
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=60, t=40, pad=4),
        xaxis=dict(
            title_text='targets',
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
            range=[0,11],
        ),
        yaxis=dict(
            title_text='CV (sigma/mean)',
            tickformat='.2f',
            range=[0,0.9],
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# single damage roll
targets = list(range(1, 11))
cv = [save_sigma(fail_state, save_state, n)/save_mean(fail_state, save_state, n) for n in targets]
fig.add_trace(go.Scatter(
    x=targets, 
    y=cv,
    name=f'single damage roll',
    mode='lines+markers',
    hovertemplate=
        f'<b>spell</b><br>'+
        'targets %{x:.0f}<br>'+
        'CV total %{y:.2f}<extra></extra>'
))

# multiple damage rolls
targets = list(range(1, 11))
cv = [(save_sigma(fail_state, save_state, 1)/save_mean(fail_state, save_state, 1))/np.sqrt(n) for n in targets]
fig.add_trace(go.Scatter(
    x=targets, 
    y=cv,
    name=f'multiple damage rolls',
    mode='lines+markers',
    hovertemplate=
        f'<b>spell</b><br>'+
        'targets %{x:.0f}<br>'+
        'CV total %{y:.2f}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=[0,11], 
    y=2*[fail_state['sigma']/fail_state['mean']],
    showlegend=False,
    mode='lines',
    line=dict(color='black', dash='dash'),
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450, barmode='group')
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-multi-target-save-cv-zero-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-multi-target-save-cv-zero-small', style='aspect-ratio:600/500')

0.3600411499115478
